# Gcode 34

In [2]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 


In [3]:
model = AbstractModel()
model.gen=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.load_val=Param(initialize=400,mutable=True)
model.ep=Param(initialize=0,within=Reals,mutable=True)
#Scalar

Eprice  = 0.1

In [4]:
#constraint
def Pbounds(model,gen):
  return (model.Pmin[gen],model.Pmax[gen])

model.p=Var(model.gen,bounds=Pbounds,within=Reals)
model.TE=Var(within=Reals)
model.TC=Var(within=Reals)

def rule1(model,gen):
  return sum(model.p[gen] for gen in model.gen) >= model.load_val
model.c1 = Constraint(rule=rule1)

def ruleTC(model,gen):
  return model.TC == sum((model.a[gen]*model.p[gen]*model.p[gen] + 
                              model.b[gen]*model.p[gen]+model.c[gen]) for gen in model.gen) 
model.c2 = Constraint(rule=ruleTC)

def ruleTE(model,gen):
  return model.TE == sum((model.d[gen]*model.p[gen]*model.p[gen] + 
                              model.e[gen]*model.p[gen]+model.f[gen]) for gen in model.gen) 
model.c3 = Constraint(rule=ruleTE)


In [5]:
model.obj1=Objective(expr=model.TC,sense=minimize)
model.obj2=Objective(expr=model.TE,sense=minimize)


In [6]:
instance= model.create_instance("gcode33.dat")


In [14]:
instance.obj2.deactivate()
results = SolverFactory('ipopt').solve(instance).write()
minof1=instance.obj1()
maxof2=instance.obj2()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 7
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.14.14\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.047846317291259766
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [10]:
instance.obj1.deactivate()
instance.obj2.activate()
results = SolverFactory('ipopt').solve(instance).write()
maxof1=instance.obj1()
minof2=instance.obj2()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 7
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.14.14\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.09089517593383789
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [16]:
# for counter
instance.eq2= Constraint(expr= instance.TE >=instance.ep)
Nsteps = 11;
for counter in range(1,Nsteps+1):
    instance.ep = (maxof2-minof2)*(counter-1)/(Nsteps-1)+(minof2);
    results = SolverFactory('ipopt').solve(instance);
    print('c',counter)
    print('Total Cost  =   ',round(instance.TC(),2));
    print('Total Emission=  ',round(instance.TE(),2));
    #instance.p.pprint()

'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
c 1
Total Cost  =    386827.87
Total Emission=   268726.47
c 2
Total Cost  =    392073.24
Total Emission=   273272.84
c 3
Total Cost  =    398146.94
Total Emission=   278552.08
c 4
Total Cost  =    405151.93
Total Emission=   284658.17
c 5
Total Cost  =    413173.97
Total Emission=   291669.94
c 6
Total Cost  =    422267.68
Total Emission=   299637.95
c 7
Total Cost  =    432445.74
Total Emission=   308573.65
c 8
Total Cost  =    443675.47
Total Emission=   318445.08
c 9
Total Cost  =    455884.91
Total Emission=   329181.13
c 10
Total Cost  =    468976.43
Total Emission=   340683.06
c 11
Total Cost  =    482843.23
Total Emission=   352838.97


In [12]:
# for counter
instance.eq2= Constraint(expr= instance.TE >=instance.ep)
Nsteps = 11;
for counter in range(1,Nsteps+1):
    instance.ep = (maxof1-minof1)*(counter-1)/(Nsteps-1)+(minof1);
    results = SolverFactory('ipopt', executable='/content/ipopt').solve(instance);
    print('c',counter)
    print('Total Cost  =   ',round(instance.TC(),2));
    print('Total Emission=  ',round(instance.TE(),2));
    #instance.p.pprint()


'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
for solver ipopt. File with name=/content/ipopt either does not exist or it is
not executable. To skip this validation, call set_executable with
validate=False.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/pyomo/opt/base/solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pyomo/solvers/plugins/solvers/IPOPT.py", line 40, in __init__
    super(IPOPT, self).__init__(**kwds)
  File "/opt/anaconda3/lib/python3.11/site-packages/pyomo/opt/solver/shellcmd.py", line 65, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/opt/anaconda3/lib/python

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "ipopt"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: /content/ipopt
	type: ipopt
	_args: ()
	options: {}